# Gaussian Process 

### Data Training

In [11]:
import sys
import os

# Add the parent directory to the sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

import numpy as np
from elliptic_files.train_elliptic import samples_param
from elliptic_files.FEM_Solver import FEMSolver

obs, nthetas = 6, 100
thetas  = samples_param(nthetas,nparam=2)
fem_solver = FEMSolver(np.zeros(2),vert=50)
obs_points = np.linspace(0.2,0.8,obs).reshape(-1,1)
training_data = np.zeros((nthetas,obs ))

for i,theta in enumerate(thetas):
    fem_solver.theta = theta
    fem_solver.solve()
    training_data[i,:] = fem_solver.eval_at_points(obs_points).reshape(1, -1)